In [16]:
import os
import pymongo
import pandas as pd
from datetime import datetime

In [17]:
conn_str = os.getenv("MONGO_CONN_STR")
client = pymongo.MongoClient(conn_str, serverSelectionTimeoutMS=5000)

try:
    client.server_info()
except Exception:
    print("Unable to connect to the server.")

database = client.get_database()

In [18]:
cart_collection = database.get_collection("carts")
demand_collection = database.get_collection("demands")
product_collection = database.get_collection("products")
county_collection = database.get_collection("counties")


In [19]:
today = datetime.today()
tomorrow = today.replace(day=today.day + 1, hour=0, minute=0, second=0, microsecond=0)
today = today.replace(hour=0, minute=0, second=0, microsecond=0)

demands = demand_collection.find({
  "end_date": { "$gte": today, "$lt": tomorrow }
})

closed_demands = [str(demand["_id"]) for demand in demands]

In [20]:
carts = cart_collection.find({
    "state" : { "$eq" : "closed"},
    "demand_id": { "$in": closed_demands}
  })

products = product_collection.find()
counties = county_collection.find()

In [21]:
df_carts = pd.DataFrame(carts)
df_products = pd.DataFrame(products)
df_counties = pd.DataFrame(counties)

In [22]:
df_carts = df_carts.rename(columns={'_id': 'cart_id'})
df_products = df_products.rename(columns={'_id': 'product_id'})

In [23]:
df_carts.county_id = df_carts.county_id.astype(str)
df_counties._id = df_counties._id.astype(str)

df_carts_with_counties = df_carts.merge(df_counties[["_id", "name"]], left_on="county_id", right_on="_id")[["name", "products"]]

In [24]:
df_exploded = df_carts_with_counties.explode(column="products")

In [25]:
def getAtrr(row, atrribute):
  rowDict = dict(row)
  return rowDict[atrribute]

df_exploded["_id"] = df_exploded["products"].apply(lambda x: getAtrr(x, "_id"))
df_exploded["quantity"] = df_exploded["products"].apply(lambda x: getAtrr(x, "quantity"))

In [26]:
df_exploded.drop(columns=["products"], inplace=True)

In [27]:
def getProductDesc(row):
  measurementsList = list(row["measurements"])
  newList = []
  for m in measurementsList:
    newList.append(" ".join(list(m.values())))

  normsList = list(row["norms"])
  
  return row["name"] + " " + " ".join(newList) + " " + " ".join(normsList)
  
df_products["description"] = df_products.apply(lambda x: getProductDesc(x), axis=1)

In [28]:
newNames = {"name": "Município / Autarquia", "description": "Produto", "quantity": "Quantidade"}

df_products.product_id = df_products.product_id.astype(str)
df_exploded._id = df_exploded._id.astype(str)

df = df_exploded.merge(df_products[["product_id", "description"]], left_on="_id", right_on="product_id").drop(columns="_id").rename(columns=newNames)

In [29]:
df_pivot = df.pivot_table(index="Produto", values="Quantidade", columns="Município / Autarquia")
df_pivot["Total"] = df_pivot.sum(axis=1)

In [30]:
df_pivot.to_excel("Consolidado de Produtos.xlsx")

In [8]:
from datetime import datetime
format_data = "%Y-%m-%d_%H-%M-%S"
datetime.strftime(datetime.today(), format_data)

'2023-02-04_15-30-45'